In [72]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from ppopt.plot import parametric_plot
from numpy.polynomial.legendre import leggauss
import copy
from typing import Union
from collections import defaultdict

In [73]:
m = MPModeler()

u = m.add_var(name='u')
x = m.add_var(name='x')
z = m.add_var(name='z')

t1 = m.add_param(name='t1')
t2 = m.add_param(name='t2')
d1 = m.add_param(name='d1')
d2 = m.add_param(name='d2')

In [74]:
m.add_constr(2*x - 3*z + t1 - d2 == 0)
m.add_constr(x - z/2 -t1/2 +t2/2 +d1 -7*d2/2 <= u)
m.add_constr(-2*x +2*z -4*t1/3 -t2 +2*d2 +1/3<= u)
m.add_constr(-x + 5*z/2 +t1/2 -t2 -d1 +d2/2 -1 <= u)
m.add_constr(-50 <= x)
m.add_constr(0-50 <= z)
m.add_constr(0 <= t1)
m.add_constr(0 <= t2)
m.add_constr(0 <= d1)
m.add_constr(0 <= d2)
m.add_constr(t1 <= 4)
m.add_constr(t2 <= 4)
m.add_constr(d1 <= 5)
m.add_constr(d2 <= 5)

In [75]:
m.set_objective(u)

In [76]:
prob = m.formulate_problem()
prob.process_constraints()

In [77]:
solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.combinatorial)

In [86]:
def mpformulate_theta_bounds(flex_sol, num_theta:int, num_design:int, theta_bounds:list, design_bounds:list, psi_idx:int=0, theta_m:int=0):
    A0, b0, F0 = np.empty((len(flex_sol), num_theta)), np.empty((len(flex_sol), 1)), np.empty((len(flex_sol), num_design))
    num_cr = len(flex_sol.critical_regions)
    for i, region in enumerate(flex_sol.critical_regions):
        A0[i] = region.A[psi_idx,:num_theta]
        b0[i] = -region.b[psi_idx]
        F0[i] = -region.A[psi_idx, num_theta:num_theta+num_design]
    
    row1_block = np.hstack([block for i in range(theta_m, num_theta) for block in (A0[:, [i]], np.zeros((num_cr, 1)))])
    row2_block = np.hstack([block for i in range(theta_m, num_theta) for block in (np.zeros((num_cr, 1)), A0[:, [i]])])
    bound_row = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))])
    A = np.vstack([row1_block, row2_block, bound_row, -np.eye(2*(num_theta-theta_m)), np.eye(2*(num_theta-theta_m))])
    # print(f'A: {A}')
    # print(f'A.shape: {A.shape}')
    
    x_lb = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][0]] * 2])
    x_ub = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][1]] * 2])
    b = np.vstack([b0, b0, np.zeros((1,1)), x_lb.reshape(-1,1), x_ub.reshape(-1,1)])
    # print(f'b: {b}')
    # print(f'b.shape: {b.shape}')
    
    F = np.vstack([F0, F0, np.zeros((1,num_design)), np.zeros((4*(num_theta-theta_m), num_design))])
    if theta_m > 0:
        F_lltheta = np.hstack([A0[:, [i]] for i in range(theta_m)])
        # print(f'F_lltheta: {F_lltheta}')
        F = np.hstack([np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))]), F])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    
    H = np.zeros((2*(num_theta-theta_m), theta_m+num_design))
    # print(f'H:{H}')
    # print(f'H.shape: {H.shape}')
    
    A_t = np.vstack([-np.eye(theta_m+num_design), np.eye(theta_m+num_design)])
    # print(f'A_t:{A_t}')
    # print(f'A_t.shape: {A_t.shape}')
    
    theta_lb = np.array([theta_bounds[i][0] for i in range(theta_m)] + [j[0] for j in design_bounds]).reshape(-1,1)
    theta_ub = np.array([theta_bounds[i][1] for i in range(theta_m)] + [j[1] for j in design_bounds]).reshape(-1,1)
    b_t = np.vstack([theta_lb, theta_ub])
    # print(f'b_t:{b_t}')
    # print(f'b_t.shape: {b_t.shape}')
    
    c = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))]).reshape(-1,1)
    # print(f'c:{c}')
    # print(f'c.shape: {c.shape}')
    
    return A, b, c, H, A_t, b_t, F

In [87]:
# A, b, c, H, A_t, b_t, F = mpformulate_theta_bounds(flex_sol=solution, num_theta=2 ,num_design=2, theta_bounds=[(0,4),(0,4)], design_bounds=[(0,5), (0,5)], theta_m=0)

In [88]:
# prob_test = MPLP_Program(A=A, b=b, c=c, H=H, A_t=A_t, b_t=b_t, F=F)
# prob_test.process_constraints()

In [89]:
# solution_test = solve_mpqp(problem=prob_test, algorithm=mpqp_algorithm.geometric)

In [90]:
# solution_test

In [91]:
# parametric_plot(solution_test, show=True)

In [92]:
theta_bounds=[(0,4),(0,4)]
design_bounds=[(0,5), (0,5)]
nt = 2
nd = 2

In [93]:
theta_bound_dict = defaultdict(dict)
for i in range(nt):
    A, b, c, H, A_t, b_t, F = mpformulate_theta_bounds(flex_sol=solution_flexibility, num_theta=nt ,num_design=nd, theta_bounds=theta_bounds, design_bounds=design_bounds, theta_m=i)
    prob = MPLP_Program(A=A, b=b, c=c, H=H, A_t=A_t, b_t=b_t, F=F)
    prob.process_constraints()
    solution = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
    theta_bound_dict[f't{i}'] = solution

Using a found active set [6, 9, 11, 12]
Using a found active set [6, 7]


In [94]:
solutions_list = [sol for key, sol in theta_bound_dict.items()]

In [96]:
def gauss_legendre_between_bounds(expr_coeffs: np.ndarray, n_gl: int, max_idx: int = 0, min_idx: int = 1):
    """
    Generate n Gauss–Legendre quadrature points and weights between min and max bounds
    defined by two linear expressions.

    Parameters:
        expr_coeffs (np.ndarray): 2xD array. Row 0 = max point coefficients, Row 1 = min.
        n (int): Number of quadrature points.

    Returns:
        points (np.ndarray): (n, D) array of quadrature points.
        weights (np.ndarray): (n,) array of weights.
    """
    if expr_coeffs.shape[0] != 2:
        raise ValueError("expr_coeffs must have two rows")

    max_coeffs = expr_coeffs[max_idx]
    min_coeffs = expr_coeffs[min_idx]

    # Get Gauss–Legendre points and weights on [-1, 1]
    nodes, weights = leggauss(n_gl)
    weights = weights.reshape(-1,1)
    
    # Affine transformation to domain [min_coeffs, max_coeffs]
    points = 0.5 * (np.outer((nodes + 1), max_coeffs) + np.outer((1 - nodes), min_coeffs))

    # Adjust weights to match new domain
    weights = 0.5 * weights@(max_coeffs - min_coeffs).reshape(1,-1)

    return points, weights

# # Example: 2D line segment from [0,0] (min) to [1,1] (max)
# # coeffs = np.array([
# #     [0, 0, 4],  # max point
# #     [0, 0, 0],  # min point
# # ])
# 
# coeffs = np.array([
#     [0, 0, 4],
#     [0.75,-1.5,-1.25]
# ])
# 
# # coeffs = np.array([
# #     [0.000000,0.0,0.0,4.000000],
# #     [-2.666667,2.0,-4.0,0.666667]
# # ])
# 
# points, weights = gauss_legendre_between_bounds(expr_coeffs=coeffs, n_gl=5)
# 
# print("Points Mapped:\n", points)
# print("Weights Mapped:\n", weights)

In [97]:
def joint_pdf(theta:list):
    return (2/np.pi)*np.exp(-2*((theta[0]-2)**2 + (theta[1]-2)**2))

In [98]:
def get_quadrature_points(solution, nq: int, t_vector: np.ndarray):
    # Augment t_vector once
    t_vector_aug = np.append(t_vector, 1).reshape(-1, 1)

    for region in solution.critical_regions:
        if region.is_inside(t_vector):
            coeffs = np.concatenate([region.A, region.b], axis=1)[:2, :]
            qpoints, qweights = gauss_legendre_between_bounds(expr_coeffs=coeffs, n_gl=nq)
            return coeffs[0] @ t_vector_aug, coeffs[1] @ t_vector_aug, qpoints @ t_vector_aug, qweights @ t_vector_aug

    raise ValueError("No region found that contains the given t_vector.")


In [99]:
# def calculate_stocflexibility(sols: list, nq: int, d_vector: np.ndarray) -> float:
#     # Get first-stage quadrature points and weights
#     t1max, t1min, t1_points, t1_weights = get_quadrature_points(solution=sols[0], nq=nq, t_vector=d_vector)
#     t1_points = t1_points.flatten()
#     t1_weights = t1_weights.flatten()
# 
#     outer_sum = 0.0
# 
#     for i in range(nq):
#         v1 = t1_points[i]
#         w1 = t1_weights[i]
# 
#         # Second-stage quadrature points and weights using updated vector
#         t2max, t2min, t2_points, t2_weights = get_quadrature_points(solution=sols[1], nq=nq,t_vector=np.block([np.array([v1]), d_vector]))
#         t2_points = t2_points.flatten()
#         t2_weights = t2_weights.flatten()
#         # print(f't2points: {t2_points} for t1:{v1}')
#         # print(f't2weights: {t2_weights} for t1:{v1}')
# 
#         # Inner quadrature
#         inner_sum = np.sum([w2 * joint_pdf(theta=[v1, v2]) for v2, w2 in zip(t2_points, t2_weights)])
#         outer_sum += w1 * inner_sum
#     
#     # print(f't1points: {t1_points}')
#     # print(f't1weights: {t1_weights}')
#     return outer_sum

In [103]:
def calculate_stocflexibility(sols: list, nq: Union[int, list], d_vector: np.ndarray) -> float:
    """
    Calculates the stochastic flexibility index via nested Gauss–Legendre quadrature.

    :param sols: list of solution objects (one per stage)
    :param nq: int or list of ints; if int, used for all levels; if list, should match len(sols)
    :param d_vector: base vector used to condition quadrature
    :return: stochastic flexibility index
    """

    # Validate nq if it's a list
    if isinstance(nq, list):
        if len(nq) != len(sols):
            raise ValueError("If nq is a list, it must have the same length as sols")

    def recurse(level: int, theta_prev: list, weight_prev: float) -> float:
        """
        Recursive inner function to compute nested quadrature.
        """
        if level == len(sols):
            return weight_prev * joint_pdf(theta_prev)

        # Use nq[level] if nq is a list, otherwise use scalar nq
        nql = nq[level] if isinstance(nq, list) else nq

        t_vector = np.block([np.array(theta_prev), d_vector])
        # print(f't_vector:{t_vector}')
        _, _, t_points, t_weights = get_quadrature_points(solution=sols[level], nq=nql, t_vector=t_vector)

        t_points = t_points.flatten()
        t_weights = t_weights.flatten()
        
        # print(f'theta_prev: {theta_prev}')
        return sum(
            recurse(level + 1, theta_prev + [v], weight_prev * w)
            for v, w in zip(t_points, t_weights)
        )

    return recurse(level=0, theta_prev=[], weight_prev=1.0)


In [104]:
sf_idx = calculate_stocflexibility(sols=solutions_list, nq=8, d_vector=np.array([3.2, 0.4]))

theta_prev: []
theta_prev: [0.6184999975417473]
theta_prev: [0.9007503264614916]
theta_prev: [1.3684565928943302]
theta_prev: [1.9585752416950024]
theta_prev: [2.5914247583049943]
theta_prev: [3.181543407105667]
theta_prev: [3.6492496735385056]
theta_prev: [3.93150000245825]


In [102]:
print(f'Stochastic Flexibility Index: {sf_idx:.4}')

Stochastic Flexibility Index: 0.8981
